## Лабораторная работа №2

Задание 1 . Провести классификацию найденного датасета, методом машины опорных векторов. В формате *Markdown* писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from imblearn.under_sampling import RandomUnderSampler

In [19]:
data = pd.read_csv('classified_data.csv')
print(data)


             ID  Loan Amount  Funded Amount  Funded Amount Investor  Term  \
0      65087372        10000          32236             12329.36286    59   
1       1450153         3609          11940             12191.99692    59   
2       1969101        28276           9311             21603.22455    59   
3       6651430        11170           6954             17877.15585    59   
4      14354669        16890          13226             13539.92667    59   
...         ...          ...            ...                     ...   ...   
67458  16164945        13601           6848             13175.28583    59   
67459  35182714         8323          11046             15637.46301    59   
67460  16435904        15897          32921             12329.45775    59   
67461   5300325        16567           4975             21353.68465    59   
67462  65443173        15353          29875             14207.44860    59   

      Batch Enrolled  Interest Rate Grade Sub Grade Employment Duration  ..

In [25]:
label_encoder = LabelEncoder()

# data['Grade Encoded'] = label_encoder.fit_transform(data['Grade'])
# data['Sub Grade Encoded'] = label_encoder.fit_transform(data['Sub Grade'])
# data['Employment Duration Encoded'] = label_encoder.fit_transform(data['Employment Duration'])
# data['Verification Status Encoded'] = label_encoder.fit_transform(data['Verification Status'])
# data['Loan Title Encoded'] = label_encoder.fit_transform(data['Loan Title'])
# data['Initial List Status Encoded'] = label_encoder.fit_transform(data['Initial List Status'])

non_numeric_columns = data.select_dtypes(exclude=['number']).columns.tolist()

data = data.drop(columns=non_numeric_columns)
# data = data.drop("ID", axis=1)
data = data.drop("Accounts Delinquent", axis=1)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67463 entries, 0 to 67462
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan Amount                   67463 non-null  int64  
 1   Funded Amount                 67463 non-null  int64  
 2   Funded Amount Investor        67463 non-null  float64
 3   Term                          67463 non-null  int64  
 4   Interest Rate                 67463 non-null  float64
 5   Home Ownership                67463 non-null  float64
 6   Debit to Income               67463 non-null  float64
 7   Delinquency - two years       67463 non-null  int64  
 8   Inquires - six months         67463 non-null  int64  
 9   Open Account                  67463 non-null  int64  
 10  Public Record                 67463 non-null  int64  
 11  Revolving Balance             67463 non-null  int64  
 12  Revolving Utilities           67463 non-null  float64
 13  T

In [28]:
X = data.drop('Loan Status', axis=1) 
y = data['Loan Status']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

under_sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_test)

print(pd.Series(y_resampled).value_counts())
print(y_test.unique())

param_grid = {'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
              'C': [0.001, 0.01, 0.1, 1, 10],
              'degree': [1, 2, 3, 4, 5, 6, 7]} 

svm_grid = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

svm_grid.fit(X_resampled, y_resampled)

best_params = svm_grid.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']
best_degree = best_params['degree']

print(f"Лучшие параметры: Ядро = {best_kernel}, C = {best_C}, Degree = {best_degree}")

0    1853
1    1853
Name: Loan Status, dtype: int64
[0 1]
Лучшие параметры: Ядро = sigmoid, C = 1, Degree = 1


In [29]:
svm_model = SVC(kernel=best_kernel, C=best_C) 

svm_model.fit(X_resampled, y_resampled)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division = 1)

print(f'Точность модели: {accuracy}')
print(report)

stratified_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm_model, X_resampled, y_resampled, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')

Точность модели: 0.5615396017589802
              precision    recall  f1-score   support

           0       0.91      0.57      0.70     18386
           1       0.09      0.43      0.15      1853

    accuracy                           0.56     20239
   macro avg       0.50      0.50      0.43     20239
weighted avg       0.83      0.56      0.65     20239

Средняя точность перекрестной проверки: 0.4997301672962763
